# Laboratory: Fundamentals of NF and Sensitivity Calculations

## 1. Objectives

- **Noise Figure (NF) Concept:** Understand how individual components, both active (amplifiers, mixers) and passive (filters), degrade the Signal-to-Noise Ratio (SNR) of a system.

- **Cascaded NF (Friis Formula):**  Analyze how the order of components and the gain of the early stages dictate the overall system sensitivity.

- **Design Trade-offs:** Identify why the Low Noise Amplifier (LNA) is the most critical stage in a receiver front-end.

## 2. Theoretical Background

### RF Receiver Sensitivity

The sensitivity of an RF receiver is a "power budget" that considers the physical environment, hardware limitations, and modulation scheme requirements.

Every stage generates thermal noise at temperatures above absolute zero. The noise power for bandwidth \(B\) is:

$$ P_N = k T_0 B $$

In dBm, for room temperature (\(290 \text{K}\)):

$$ P_{N(\text{dBm})} = -174 \text{ dBm/Hz} + 10 \log_{10}(B) $$

Also known as detection SNR, it is determined by modulation type and target Bit Error Rate (BER), defining how far the signal must be above the noise to be decoded. Assuming that every receiver has its own NF the sensitivity is related to these variables by:

$$
S_{\text{dBm}} = P_{N(\text{dBm})} + NF_{\text{dB}} + SNR_{min(\text{dB})}
$$


### Friis Formula for Cascaded NF

The core of this analysis is the **Friis Formula**. It proves that the noise contribution of each subsequent stage is reduced by the total gain of the preceding stages:

$$
F_{total} = F_1 + \frac{F_2 - 1}{G_1} + \frac{F_3 - 1}{G_1 G_2} + \dots + \frac{F_n - 1}{G_1 G_2 \dots G_{n-1}}
$$

**Note**: Calculations must be performed in linear units (Noise Factor \(F\) and Linear Gain \(G\)) before converting the final result to decibels:

$$
NF = 10 \log_{10} F
$$

## 3. How to Run the Examples

This notebook is interactive. To interact with the simulations:
- *Initialization:* Execute the "Init Packages" cell below first to load `numpy`, `matplotlib`, and `ipywidgets`.

- *Execution:* Select a code cell and press **Shift + Enter** or click the **Run** button in the toolbar.

- Interaction:* Use the sliders to adjust Gain, Noise Figure, and Input Power in real-time.


In [3]:
# Check if packages are installed and install if needed
import subprocess
import sys

def install_package(package):
    try:
        # Try to import the package
        __import__(package)
        print(f"{package} is already installed.")
    except ImportError:
        # If the package is not installed, install it
        print(f"{package} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# List of packages to check and install if needed
packages = ["numpy", "matplotlib", "ipywidgets", "scipy"]

for package in packages:
    install_package(package)

# Importing after ensuring packages are installed
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets, interactive_output, VBox, HBox
from IPython.display import display, clear_output
from scipy.fft import fft, fftfreq, fftshift


numpy is already installed.
matplotlib is already installed.
ipywidgets is already installed.
scipy is already installed.


In [13]:
def plot_noise_power_levels(mode, gain_db, nf_db, p_in_dbm):
    # --- 1. Constants ---
    bw = 1e6  # 1 MHz Bandwidth
    t0 = 290
    k = 1.38e-23
    
    # Thermal Noise Floor (Pn = kTB) 
    # At 290K and 1MHz BW, Pn is approximately -114 dBm
    thermal_noise_dbm = round(10 * np.log10(k * t0 * bw) + 30)
    
    # Input Calculations
    input_snr = p_in_dbm - thermal_noise_dbm
    
    # Define Gain and NF based on mode
    if mode == 'Passive (Attenuator)':
        actual_gain = -abs(gain_db)
        actual_nf = abs(gain_db) 
    else:
        actual_gain = gain_db
        actual_nf = nf_db
        
    # Output Calculations
    output_signal = p_in_dbm + actual_gain
    # Noise at output = Input Noise + Gain + Noise Figure
    output_noise = thermal_noise_dbm + actual_gain + actual_nf
    output_snr = output_signal - output_noise

    # --- 2. Plotting ---
    plt.figure(figsize=(10, 6))
    
    # Stages for X-axis
    stages = [0, 1]
    
    # Plot Signal Lines
    plt.plot(stages, [p_in_dbm, output_signal], color='green', marker='o', lw=3, label='Signal Power (dBm)')
    # Plot Noise Lines
    plt.plot(stages, [thermal_noise_dbm, output_noise], color='red', marker='s', lw=3, label='Noise Power (dBm)')
    
    # Shading the SNR gap
    plt.fill_between(stages, [thermal_noise_dbm, output_noise], [p_in_dbm, output_signal], 
                     color='blue', alpha=0.1, label='SNR Gap')

    # Annotations for Values
    plt.text(0, p_in_dbm + 2, f'{p_in_dbm} dBm', color='green', fontweight='bold', ha='right')
    plt.text(1, output_signal + 2, f'{output_signal} dBm', color='green', fontweight='bold', ha='left')
    plt.text(0, thermal_noise_dbm - 5, f'{thermal_noise_dbm:.1f} dBm', color='red', ha='right')
    plt.text(1, output_noise - 5, f'{output_noise:.1f} dBm', color='red', ha='left')
    
    # SNR Labels
    plt.text(0, (p_in_dbm + thermal_noise_dbm)/2, f'SNR In: {input_snr:.1f} dB', 
             bbox=dict(facecolor='white', alpha=0.8), ha='center')
    plt.text(1, (output_signal + output_noise)/2, f'SNR Out: {output_snr:.1f} dB', 
             bbox=dict(facecolor='white', alpha=0.8), ha='center')

    # Formatting
    plt.title(f"{mode}\nSNR Degradation = NF = {actual_nf:.2f} dB", fontsize=14)
    plt.xticks([0, 1], ['Input', 'Output'])
    plt.ylabel("Power (dBm)")
    plt.ylim(thermal_noise_dbm - 15, max(p_in_dbm, output_signal) + 20)
    plt.grid(True, linestyle=':', alpha=0.6)
    plt.legend(loc='upper left')
    
    plt.show()

# --- UI Controls ---
mode_w = widgets.Dropdown(options=['Active (Amplifier)', 'Passive (Attenuator)'], value='Active (Amplifier)', description='Stage:')
gain_w = widgets.IntSlider(min=0, max=40, value=20, description='Gain | Loss (dB)')
nf_w = widgets.FloatSlider(min=0, max=15, value=5.0, step=0.5, description='NF (dB)')
pin_w = widgets.IntSlider(min=-100, max=-40, value=-70,step=0.5, description='Pin (dBm)')

ui = VBox([HBox([mode_w, pin_w]), HBox([gain_w, nf_w])])
out = interactive_output(plot_noise_power_levels, {'mode': mode_w, 'gain_db': gain_w, 'nf_db': nf_w, 'p_in_dbm': pin_w})

display(ui, out)

Output()

In [4]:
import ipywidgets as widgets
from IPython.display import display, HTML
import numpy as np
import matplotlib.pyplot as plt

# Constants
k = 1.38e-23  # Boltzmann constant [J/K]
T0 = 290      # Reference temperature [K]
SENSITIVITY = -102.0  # GSM Spec fixed at -102 dBm

# Enable inline plotting
%matplotlib inline

def calculate_gsm_nf(bw_khz, snr_db):
    # Convert bandwidth to Hz
    bw_hz = bw_khz * 1e3
    
    # 1. Thermal Noise Floor (dBm) = 10*log10(kTB) + 30
    thermal_noise_dbm = 10 * np.log10(k * T0 * bw_hz) + 30

    
    # 2. Max NF = Sensitivity - Thermal Noise - Required SNR
    max_nf = SENSITIVITY - thermal_noise_dbm - snr_db

    # --------- TEXT OUTPUT ----------
    output_html = f"""
    <h3>GSM Receiver NF Budget</h3>
    <b>Thermal Noise:</b> {thermal_noise_dbm:.2f} dBm<br>
    <b>Required SNR:</b> {snr_db:.2f} dB<br>
    <b>Sensitivity:</b> {SENSITIVITY:.2f} dBm<br>
    <h2 style="color:red;">Max NF = {max_nf:.2f} dB</h2>
    """
    display(HTML(output_html))

    # --------- GRAPHICAL BAR ----------
    plt.figure(figsize=(8,2))

    components = [thermal_noise_dbm, snr_db, max_nf]
    labels = ['Thermal Noise', 'SNR', 'NF']
    colors = ['steelblue', 'seagreen', 'indianred']

    left = 0
    for comp, label, color in zip(components, labels, colors):
        plt.barh(0, comp, left=left, color=color, label=label)
        left += comp

    # Add sensitivity line
    plt.axvline(SENSITIVITY, linestyle='--', color='black', label='Sensitivity')
    plt.yticks([])  # Remove y-axis labels
    plt.xlabel("Power (dB)")
    plt.legend()
    plt.title("Receiver Sensitivity Budget")
    plt.show()

# Create Sliders
bw_slider = widgets.FloatSlider(
    value=200.0, min=10.0, max=1000.0, step=10.0, 
    description='BW (kHz):', continuous_update=True
)

snr_slider = widgets.FloatSlider(
    value=9.0, min=0.0, max=30.0, step=0.5, 
    description='SNR (dB):', continuous_update=True
)

# Layout
ui = widgets.HBox([bw_slider, snr_slider])
out = widgets.interactive_output(calculate_gsm_nf, {'bw_khz': bw_slider, 'snr_db': snr_slider})

display(ui, out)


Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '<IPython.core.display.HTML object>', '…

In [7]:
def plot_5_stage_cascaded(g1, n1, g2, n2, g3, n3, g4, n4, g5, n5, p_in_dbm):
    # --- 1. Constants ---
    bw = 1e6  # 1 MHz Bandwidth
    t0 = 290
    k = 1.38e-23
    thermal_noise_dbm = np.ceil(10 * np.log10(k * t0 * bw) + 30)
    
    # --- 2. Arrays for Stages ---
    gs = [g1, g2, g3, g4, g5]
    nfs = [n1, n2, n3, n4, n5]
    
    # Convert to Linear for Friis Equation
    G_lin = [10**(g/10) for g in gs]
    F_lin = [10**(n/10) for n in nfs]
    
    # --- 3. Friis's Formula (Cumulative) ---
    f_cum_db = []
    current_f = 0
    current_g_prod = 1
    
    for i in range(len(gs)):
        if i == 0:
            current_f = F_lin[i]
        else:
            current_g_prod *= G_lin[i-1]
            current_f += (F_lin[i] - 1) / current_g_prod
        f_cum_db.append(10 * np.log10(current_f))
    
    # Final linear Noise Factor for the title
    sys_f_lin = current_f
    
    # --- 4. Power Level Tracking ---
    stages_idx = ['In', 'RF Filter', 'LNA', 'Mixer', 'IF Filter', 'IF Amp']
    sig_levels = [p_in_dbm]
    noise_levels = [thermal_noise_dbm]
    
    cum_gain = 0
    for i in range(len(gs)):
        cum_gain += gs[i]
        sig_levels.append(p_in_dbm + cum_gain)
        noise_levels.append(thermal_noise_dbm + cum_gain + f_cum_db[i])
        
    # --- 5. Plotting ---
    plt.figure(figsize=(14, 7))
    x = np.arange(len(stages_idx))
    
    plt.plot(x, sig_levels, color='green', marker='o', lw=3, label='Signal Power (dBm)')
    plt.plot(x, noise_levels, color='red', marker='s', lw=3, label='Noise Floor (dBm)')
    
    plt.fill_between(x, noise_levels, sig_levels, color='blue', alpha=0.1, label='SNR')

    # Value Labels
    for i in range(len(x)):
        plt.text(x[i], sig_levels[i] + 2, f'{sig_levels[i]:.2f}', color='green', fontweight='bold', ha='center')
        plt.text(x[i], noise_levels[i] - 5, f'{noise_levels[i]:.2f}', color='red', ha='center')
        snr = sig_levels[i] - noise_levels[i]
        plt.text(x[i], (sig_levels[i]+noise_levels[i])/2, f'SNR:\n{snr:.2f}dB', 
                 ha='center', va='center', bbox=dict(facecolor='white', alpha=0.7, edgecolor='none'))

    # Updated Title with Linear Noise Factor (F) and Noise Figure (dB)
    sys_nf_db = f_cum_db[-1]
    sys_gain = sum(gs)
    plt.title(f"5-Stage Receiver Chain Analysis\nSystem Gain: {sys_gain:.2f} dB | System Noise Factor (F): {sys_f_lin:.2f} | System NF: {sys_nf_db:.2f} dB", 
              fontsize=14, fontweight='bold')
    
    plt.xticks(x, stages_idx)
    plt.ylabel("Power (dBm)")
    plt.grid(True, linestyle=':', alpha=0.6)
    plt.ylim(thermal_noise_dbm - 20, max(sig_levels) + 20)
    plt.legend(loc='upper left')
    plt.show()

# --- UI Controls with Table Starting Points ---
s1_g = widgets.FloatSlider(min=-4, max=-1, value=-2.3, step=0.1, description='G1 (RF Filt)')
s1_n = widgets.FloatSlider(min=1, max=4, value=2.3, step=0.1, description='NF1')

s2_g = widgets.FloatSlider(min=5, max=30, value=20.5, step=0.5, description='G2 (LNA)')
s2_n = widgets.FloatSlider(min=0.5, max=10, value=0.9, step=0.1, description='NF2')

s3_g = widgets.FloatSlider(min=-15, max=-5, value=-7.3, step=0.1, description='G3 (Mixer)')
s3_n = widgets.FloatSlider(min=2, max=15, value=7.0, step=0.1, description='NF3')

s4_g = widgets.FloatSlider(min=-4, max=-1, value=-3.5, step=0.1, description='G4 (IF Filt)')
s4_n = widgets.FloatSlider(min=1, max=4, value=3.5, step=0.1, description='NF4')

s5_g = widgets.FloatSlider(min=0, max=30, value=22.0, step=0.5, description='G5 (IF Amp)')

s5_n = widgets.FloatSlider(min=1, max=15, value=11, step=0.1, description='NF5')

pin = widgets.FloatSlider(min=-110, max=-40, value=-90.0, step=1, description='Pin (dBm)')

ui = VBox([
    HBox([pin]),
    HBox([s1_g, s1_n]), HBox([s2_g, s2_n]),
    HBox([s3_g, s3_n]), HBox([s4_g, s4_n]),
    HBox([s5_g, s5_n])
])

out = interactive_output(plot_5_stage_cascaded, {
    'g1': s1_g, 'n1': s1_n, 'g2': s2_g, 'n2': s2_n,
    'g3': s3_g, 'n3': s3_n, 'g4': s4_g, 'n4': s4_n,
    'g5': s5_g, 'n5': s5_n, 'p_in_dbm': pin
})

display(ui, out)

Output()

## 3. Sensitivity

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import VBox, HBox, interactive_output

def plot_sensitivity_with_voltage(temp_k, bw_hz, snr_req, g1, n1, g2, n2, g3, n3, g4, n4, g5, n5):
    # --- 1. Constants & Noise Floor ---
    k = 1.38e-23
    therm_noise_density = 10 * np.log10(k * temp_k) + 30
    pn_in = therm_noise_density + 10 * np.log10(bw_hz)
    
    # --- 2. Friis's Formula ---
    gs, nfs = [g1, g2, g3, g4, g5], [n1, n2, n3, n4, n5]
    G_lin = [10**(g/10) for g in gs]
    F_lin = [10**(n/10) for n in nfs]
    
    f_total = F_lin[0]
    g_acc = 1
    for i in range(1, len(F_lin)):
        g_acc *= G_lin[i-1]
        f_total += (F_lin[i] - 1) / g_acc
    sys_nf_db = 10 * np.log10(f_total)
    
    # --- 3. Sensitivity & MDS ---
    mds = pn_in + sys_nf_db
    sensitivity_dbm = mds + snr_req
    
    # --- 4. Voltage Conversion (assuming 50 Ohm) ---
    p_watts = 10**((sensitivity_dbm - 30) / 10)
    v_rms = np.sqrt(p_watts * 50)
    
    
    # Formatting for display
    if v_rms < 1e-6:
        v_str = f"{v_rms*1e9:.2f} nV"
    elif v_rms < 1e-3:
        v_str = f"{v_rms*1e6:.2f} \u03bcV"
    else:
        v_str = f"{v_rms*1e3:.2f} mV"

    # --- 5. Plotting ---
    plt.figure(figsize=(12, 6))
    levels = [pn_in, mds, sensitivity_dbm]
    labels = [f'Noise Floor ({temp_k}K)', 'MDS', 'Sensitivity (S)']
    colors = ['#bdc3c7', '#f39c12', '#e74c3c']
    
    plt.barh(labels, levels, color=colors, alpha=0.8)
    for i, v in enumerate(levels):
        plt.text(v + 1, i, f'{v:.2f} dBm', va='center', fontweight='bold')
        
    plt.title(f"System Sensitivity: {sensitivity_dbm:.2f} dBm ({v_str} @ 50\u03a9)\n"
              f"System NF: {sys_nf_db:.2f} dB | BW: {bw_hz/1e6:.1f} MHz", fontsize=14, fontweight='bold')
    plt.xlabel("Power Level (dBm)")
    plt.xlim(pn_in - 15, 0)
    plt.grid(axis='x', linestyle=':', alpha=0.6)
    plt.tight_layout()
    plt.show()

# --- UI Controls ---
temp_slide = widgets.IntSlider(value=298, min=10, max=500, step=2, description='Temp (K)', layout={'width': '32%'})
bw_slide = widgets.FloatSlider(value=75e6, min=50e6, max=100e6, step=5e6, description='BW (Hz)', layout={'width': '32%'})
snr_slide = widgets.FloatSlider(min=0, max=100, value=78, step=2, description='SNR (dB)', layout={'width': '32%'})

# Component Sliders (Standard 50 Ohm Receiver values)
s1 = [widgets.FloatSlider(min=-4, max=-1, value=-2.3, description='G1'), widgets.FloatSlider(min=1, max=4, value=2.3, description='NF1')]
s2 = [widgets.FloatSlider(min=5, max=30, value=20.5, description='G2'), widgets.FloatSlider(min=0.5, max=5, value=0.9, description='NF2')]
s3 = [widgets.FloatSlider(min=-15, max=-5, value=-7.3, description='G3'), widgets.FloatSlider(min=2, max=15, value=7.0, description='NF3')]
s4 = [widgets.FloatSlider(min=-4, max=-1, value=-3.5, description='G4'), widgets.FloatSlider(min=1, max=4, value=3.5, description='NF4')]
s5 = [widgets.FloatSlider(min=0, max=30, value=22.0, description='G5'), widgets.FloatSlider(min=1, max=15, value=11.0, description='NF5')]

ui = VBox([
    HBox([temp_slide, bw_slide, snr_slide]),
    HBox([s1[0], s1[1]]), HBox([s2[0], s2[1]]),
    HBox([s3[0], s3[1]]), HBox([s4[0], s4[1]]),
    HBox([s5[0], s5[1]])
])

out = interactive_output(plot_sensitivity_with_voltage, {
    'temp_k': temp_slide, 'bw_hz': bw_slide, 'snr_req': snr_slide,
    'g1': s1[0], 'n1': s1[1], 'g2': s2[0], 'n2': s2[1],
    'g3': s3[0], 'n3': s3[1], 'g4': s4[0], 'n4': s4[1], 'g5': s5[0], 'n5': s5[1]
})

display(ui, out)

Output()